In [ ]:
import io

import altair as alt
import pandas as pd
from perspective import Table, PerspectiveWidget, Aggregate, Sort, Plugin
import yaml
import quilt3

In [ ]:
%%capture
pkg = quilt3.Package.browse('akarve/smartreports', registry='s3://searchminimal')

In [ ]:
ns = pd.read_csv(io.BytesIO(pkg["data/ns.csv"].get_bytes()))
sf = pd.read_csv(io.BytesIO(pkg["data/sf.csv"].get_bytes()))

colmeta = yaml.safe_load(pkg["meta/columns.yaml"].get_bytes())
    
models = {m['name']: m for m in colmeta['models']}
tables = {m: pd.json_normalize(models[m]['columns']) for m in models}
# each table should know which model it came from
for t in tables:
    tables[t]['model'] = t

In [ ]:
# clean up dates
for c in ns.columns:
    if 'date' in c.lower():
        ns[c] = pd.to_datetime(ns[c])
        
ns = ns.sort_values(["Order Date"]).reset_index()

In [ ]:
for col in ["Order", "Fulfill"]:
    c = alt.Chart(ns).mark_bar().encode(
        x=f"{col} Date:T",
        y=f"sum({col} Quantity):Q",
        color="Name:N",
        tooltip=[f"Order Date:T", "Order Quantity:Q", "Name:N", "Cumulative Orders:Q"]
    )
    prefix = col.lower()
    c.save(f"{prefix}_by_date.json")

# Order fulfillment

In [ ]:
PerspectiveWidget(
    ns
)

# Branch tracking

In [ ]:
PerspectiveWidget(
    sf
)

# Data dictionary

In [ ]:
PerspectiveWidget(
    pd.concat(tables.values(), keys=tables.keys(), ignore_index=True)
)